In [0]:
from pyspark.sql import SparkSession, functions as F, types as T, Window
from delta.tables import DeltaTable
import os
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql.window import Window

In [0]:
from pyspark.sql import types as T

# Nome da tabela de destino no Catálogo Unity
catalog_table = "production.refined.d_quartos"

# Verifica se a tabela já existe antes de tentar criá-la
if not spark.catalog.tableExists(catalog_table):
    print(f"A tabela {catalog_table} não existe. Criando a estrutura...")
    
    # Define o schema para a dimensão de quartos
    schema = T.StructType([
        # Chave primária (hash da chave de negócio composta: hotel_id + quarto_id)
        T.StructField("pk_quarto", T.StringType(), False),
        # Chave substituta (surrogate key) sequencial e única
        T.StructField("sk_quarto", T.LongType(), False),
        
        # Chaves de negócio originais para referência
        T.StructField("hotel_id", T.IntegerType(), True),
        T.StructField("quarto_id", T.IntegerType(), True),
        
        # Atributos descritivos do quarto, conforme solicitado
        T.StructField("tipo_quarto", T.StringType(), True),
        T.StructField("capacidade_maxima", T.IntegerType(), True),
        # Usar DecimalType é a melhor prática para valores monetários
        T.StructField("preco_diaria_base", T.DecimalType(10, 2), True),

        # Metadados de controle (SCD - Slowly Changing Dimensions)
        T.StructField("start_date", T.DateType(), True),
        T.StructField("update_date", T.DateType(), True)
    ])
    
    # Cria um DataFrame vazio com o schema definido
    df_empty = spark.createDataFrame([], schema)

    # Cria a tabela Delta gerenciada no catálogo
    (
        df_empty.write
        .format("delta")
        .saveAsTable(catalog_table)
    )

    print(f"Tabela Delta '{catalog_table}' criada com sucesso.")
else:
    print(f"A tabela '{catalog_table}' já existe.")

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql.window import Window
from delta.tables import DeltaTable

# --- Parâmetros ---
catalog_table = "production.refined.d_quartos"
source_table = "production.trusted.tb_quartos" 

print("Iniciando processo de merge para a tabela:", catalog_table)

# --- 1. Carregar Tabela de Destino ---
try:
    delta_table = DeltaTable.forName(spark, catalog_table)
    df_dim_existente = delta_table.toDF()
    is_initial_load = df_dim_existente.count() == 0
except Exception as e:
    if "TABLE_OR_VIEW_NOT_FOUND" in str(e):
        is_initial_load = True
        print(f"Tabela {catalog_table} não encontrada. Assumindo carga inicial.")
    else:
        raise e

# --- 2. Carregar e Preparar Dados de Origem ---
# Seleciona as colunas da origem que correspondem à dimensão de destino.
df_source = (
    spark.read.table(source_table)
    .select(
        "hotel_id", 
        "quarto_id",
        "tipo_quarto",
        "capacidade_maxima",
        "preco_diaria_base"
    )
    .dropDuplicates(["hotel_id", "quarto_id"]) # A chave de negócio é composta
    .withColumn(
        "pk_quarto",
        # A PK é um hash da combinação de hotel_id e quarto_id
        F.sha2(F.concat_ws("||", F.col("hotel_id"), F.col("quarto_id")), 256)
    )
)

# --- 3. Identificar e Preparar Apenas os Novos Registros ---
if is_initial_load:
    df_novos_quartos = df_source
else:
    # Isola apenas os quartos que são realmente novos (não existem no destino)
    df_novos_quartos = df_source.join(
        df_dim_existente.select("pk_quarto"),
        on="pk_quarto",
        how="left_anti"
    )

if not is_initial_load and df_novos_quartos.count() == 0:
    print("Nenhum novo quarto para adicionar. Processo concluído.")
    # dbutils.notebook.exit("Nenhum novo quarto para adicionar.")

# --- 4. Gerar Chaves Surrogadas (SKs) Apenas para os Novos Registros ---
if is_initial_load:
    last_id = 0
else:
    last_id_row = df_dim_existente.agg(F.max("sk_quarto")).collect()
    last_id = last_id_row[0][0] if last_id_row and last_id_row[0][0] is not None else 0

window = Window.orderBy("pk_quarto")
df_com_novas_sks = (
    df_novos_quartos
    .withColumn("sk_quarto", (F.row_number().over(window) + last_id).cast(T.LongType()))
    .withColumn("start_date", F.current_date())
    .withColumn("update_date", F.lit(None).cast(T.DateType()))
)

# --- 5. Criar o DataFrame Final para o MERGE ---
df_final_source = df_source.join(
    df_com_novas_sks.select("pk_quarto", "sk_quarto", "start_date", "update_date"),
    "pk_quarto",
    "left"
)

# --- 6. Executar a Operação de MERGE ---
# Condição para atualizar: se os atributos de um quarto existente mudarem.
update_condition = """
    target.tipo_quarto <> source.tipo_quarto OR
    target.capacidade_maxima <> source.capacidade_maxima OR
    target.preco_diaria_base <> source.preco_diaria_base
"""

(
    delta_table.alias("target")
    .merge(
        df_final_source.alias("source"),
        "target.pk_quarto = source.pk_quarto"
    )
    .whenMatchedUpdate(
        condition=update_condition,
        set={
            "tipo_quarto": F.col("source.tipo_quarto"),
            "capacidade_maxima": F.col("source.capacidade_maxima"),
            "preco_diaria_base": F.col("source.preco_diaria_base"),
            "update_date": F.current_date()
        }
    )
    .whenNotMatchedInsert(
        values={
            "pk_quarto": F.col("source.pk_quarto"),
            "sk_quarto": F.col("source.sk_quarto"),
            "hotel_id": F.col("source.hotel_id"),
            "quarto_id": F.col("source.quarto_id"),
            "tipo_quarto": F.col("source.tipo_quarto"),
            "capacidade_maxima": F.col("source.capacidade_maxima"),
            "preco_diaria_base": F.col("source.preco_diaria_base"),
            "start_date": F.col("source.start_date"),
            "update_date": F.col("source.update_date")
        }
    )
).execute()

print(f"Merge/upsert concluído com sucesso na tabela: {catalog_table}")